In [11]:
import tkinter as tk
from tkinter import messagebox, font
import random

# --- Configuración del Juego ---
TAM_CUADRICULA = 4  # Un tablero clásico de 4x4
NUM_POZOS = 3
NUM_WUMPUS = 1
NUM_ORO = 1

# --- Colores y Fuentes ---
COLOR_FONDO = "#2c3e50"
COLOR_CELDA_OCULTA = "#34495e"
COLOR_CELDA_VISITADA = "#d5d5d5"
COLOR_CELDA_ACTUAL = "#FFFFFF" # Amarillo para el jugador
COLOR_TEXTO = "#000000"
COLOR_ORO_BRILLO = "#FFD700" # Dorado para la celda con oro

# --- Elementos del Juego (Símbolos) ---
WUMPUS = "W"
POZO = "P"
ORO = "G"
JUGADOR = "A"
VACIO = " "
HEDOR = "Hedor"
BRISA = "Brisa"
BRILLO = "Brillo"


class WumpusWorld:
    def __init__(self, root):
        self.root = root
        self.root.title("El Mundo del Wumpus")
        self.root.configure(bg=COLOR_FONDO)

        # --- Fuentes ---
        self.fuente_celda = font.Font(family="Helvetica", size=10, weight="bold")
        self.fuente_info = font.Font(family="Helvetica", size=12)

        # --- Estado del Juego ---
        self.tablero_logico = {}
        self.tablero_visitado = set()
        self.pos_jugador = (0, 0)
        self.pos_wumpus = None
        self.wumpus_vivo = True
        self.tiene_oro = False
        self.flecha_disponible = True
        self.juego_terminado = False

        # --- Frames de la GUI ---
        self.info_frame = tk.Frame(self.root, bg=COLOR_FONDO, pady=10)
        self.info_frame.pack(side=tk.TOP, fill=tk.X)
        self.info_label = tk.Label(self.info_frame, text="Bienvenido. ¡Encuentra el oro!", 
                                   font=self.fuente_info, bg=COLOR_FONDO, fg="white")
        self.info_label.pack()

        self.canvas_frame = tk.Frame(self.root, bg=COLOR_FONDO)
        self.canvas_frame.pack(side=tk.TOP, padx=10, pady=5)
        self.canvas = tk.Canvas(self.canvas_frame, 
                                width=TAM_CUADRICULA * 100, 
                                height=TAM_CUADRICULA * 100, 
                                bg=COLOR_CELDA_OCULTA)
        self.canvas.pack()

        # --- Frame de Controles (Contendrá Mover y Disparar) ---
        self.bottom_frame = tk.Frame(self.root, bg=COLOR_FONDO, pady=10)
        self.bottom_frame.pack(side=tk.BOTTOM, fill=tk.X)

        # --- Panel de Movimiento ---
        self.move_frame = tk.LabelFrame(self.bottom_frame, text="Mover", bg=COLOR_FONDO, fg="white", font=self.fuente_info, relief=tk.GROOVE)
        self.move_frame.pack(side=tk.LEFT, expand=True, fill=tk.X, padx=10)

        self.btn_arriba = tk.Button(self.move_frame, text="↑ Arriba", command=lambda: self.mover("arriba"))
        self.btn_arriba.pack(side=tk.TOP, pady=2, fill=tk.X)
        self.btn_izquierda = tk.Button(self.move_frame, text="← Izquierda", command=lambda: self.mover("izquierda"))
        self.btn_izquierda.pack(side=tk.LEFT, expand=True, fill=tk.X)
        self.btn_derecha = tk.Button(self.move_frame, text="Derecha →", command=lambda: self.mover("derecha"))
        self.btn_derecha.pack(side=tk.RIGHT, expand=True, fill=tk.X)
        self.btn_abajo = tk.Button(self.move_frame, text="↓ Abajo", command=lambda: self.mover("abajo"))
        self.btn_abajo.pack(side=tk.BOTTOM, pady=2, fill=tk.X)

        # --- Panel de Disparo ---
        self.shoot_frame = tk.LabelFrame(self.bottom_frame, text="Disparar Flecha", bg=COLOR_FONDO, fg="white", font=self.fuente_info, relief=tk.GROOVE)
        self.shoot_frame.pack(side=tk.RIGHT, expand=True, fill=tk.X, padx=10)

        self.btn_shoot_up = tk.Button(self.shoot_frame, text="↑ Arriba", command=lambda: self.disparar_flecha("arriba"))
        self.btn_shoot_up.pack(side=tk.TOP, pady=2, fill=tk.X)
        self.btn_shoot_left = tk.Button(self.shoot_frame, text="← Izquierda", command=lambda: self.disparar_flecha("izquierda"))
        self.btn_shoot_left.pack(side=tk.LEFT, expand=True, fill=tk.X)
        self.btn_shoot_right = tk.Button(self.shoot_frame, text="→ Derecha", command=lambda: self.disparar_flecha("derecha"))
        self.btn_shoot_right.pack(side=tk.RIGHT, expand=True, fill=tk.X)
        self.btn_shoot_down = tk.Button(self.shoot_frame, text="↓ Abajo", command=lambda: self.disparar_flecha("abajo"))
        self.btn_shoot_down.pack(side=tk.BOTTOM, pady=2, fill=tk.X)
        
        # Botón de Reinicio
        self.btn_reiniciar = tk.Button(self.root, text="Reiniciar Juego", command=self.iniciar_juego)
        self.btn_reiniciar.pack(side=tk.BOTTOM, pady=10)

        self.iniciar_juego()

    def iniciar_juego(self):
        """Reinicia el estado del juego y el tablero."""
        self.tablero_logico = {}
        self.tablero_visitado = set()
        self.pos_jugador = (0, 0)
        self.pos_wumpus = None
        self.wumpus_vivo = True
        self.tiene_oro = False
        self.flecha_disponible = True
        self.juego_terminado = False
        
        self.tablero_visitado.add(self.pos_jugador)

        # --- Generar Tablero Lógico ---
        celdas_disponibles = [(r, c) for r in range(TAM_CUADRICULA) for c in range(TAM_CUADRICULA) if (r, c) != (0, 0)]
        random.shuffle(celdas_disponibles)
        
        for _ in range(NUM_POZOS):
            if celdas_disponibles:
                self.tablero_logico[celdas_disponibles.pop()] = POZO

        if celdas_disponibles:
            self.pos_wumpus = celdas_disponibles.pop()
            self.tablero_logico[self.pos_wumpus] = WUMPUS

        if celdas_disponibles:
            self.tablero_logico[celdas_disponibles.pop()] = ORO
        
        self.actualizar_estado_botones(tk.NORMAL)
        self.actualizar_canvas()
        self.actualizar_info_label()

    def dibujar_cuadricula(self):
        """Dibuja la cuadrícula base en el canvas."""
        self.canvas.delete("all")
        for r in range(TAM_CUADRICULA):
            for c in range(TAM_CUADRICULA):
                x1, y1 = c * 100, r * 100
                x2, y2 = x1 + 100, y1 + 100
                pos = (r, c)
                color = COLOR_CELDA_OCULTA
                
                # Destacar la celda con oro si el jugador está en ella y no lo ha recogido
                if pos == self.pos_jugador and self.tablero_logico.get(pos) == ORO and self.tiene_oro:
                    color = COLOR_ORO_BRILLO # ¡Esta es la mejora visual!
                elif pos == self.pos_jugador:
                    color = COLOR_CELDA_ACTUAL
                elif pos in self.tablero_visitado:
                    color = COLOR_CELDA_VISITADA
                self.canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline="black")

    def dibujar_pistas(self):
        """Dibuja las pistas (Brisa, Hedor, etc.) en las celdas visitadas."""
        for pos in self.tablero_visitado:
            r, c = pos
            pistas = self.obtener_percepciones(pos)
            texto_pistas = ""
            if BRILLO in pistas:
                texto_pistas += "Brillo ✨\n"
            if HEDOR in pistas:
                texto_pistas += "Hedor 🤢\n"
            if BRISA in pistas:
                texto_pistas += "Brisa 💨\n"
            
            x, y = c * 100 + 50, r * 100 + 50
            self.canvas.create_text(x, y, text=texto_pistas, 
                                    font=self.fuente_celda, 
                                    fill=COLOR_TEXTO, 
                                    justify=tk.CENTER)
            
            if pos == self.pos_jugador:
                self.canvas.create_text(x, y - 40, text="A", 
                                        font=self.fuente_celda, 
                                        fill="red")

    def actualizar_canvas(self):
        """Redibuja todo el canvas (cuadrícula y pistas)."""
        self.dibujar_cuadricula()
        self.dibujar_pistas()

    def get_vecinos(self, pos):
        """Devuelve los vecinos válidos (no diagonales) de una posición."""
        r, c = pos
        vecinos = []
        if r > 0: vecinos.append((r - 1, c))
        if r < TAM_CUADRICULA - 1: vecinos.append((r + 1, c))
        if c > 0: vecinos.append((r, c - 1))
        if c < TAM_CUADRICULA - 1: vecinos.append((r, c + 1))
        return vecinos

    def obtener_percepciones(self, pos):
        """
        Devuelve un set de percepciones (pistas) para una posición dada.
        MODIFICADO: Ahora el Brillo se siente en casillas adyacentes.
        """
        percepciones = set()
        
        # --- LÓGICA DE REGLA ORIGINAL (OPCIONAL) ---
        # Si también quieres que el brillo se vea si ESTÁS en la misma casilla
        # (además de adyacente), descomenta esta línea:
        if self.tablero_logico.get(pos) == ORO:
            percepciones.add(BRILLO)
            
        # 2. Hedor, Brisa y Brillo (en casillas adyacentes)
        for vecino in self.get_vecinos(pos):
            contenido_vecino = self.tablero_logico.get(vecino)
            
            # Solo hay hedor si el Wumpus está vivo
            if self.wumpus_vivo and contenido_vecino == WUMPUS:
                percepciones.add(HEDOR)
                
            if contenido_vecino == POZO:
                percepciones.add(BRISA)
            
            # --- ¡AQUÍ ESTÁ EL CAMBIO! ---
            # Ahora el brillo también se detecta en casillas adyacentes
            if contenido_vecino == ORO:
                percepciones.add(BRILLO)
                
        return percepciones

    def actualizar_info_label(self):
        """Actualiza el texto de información superior."""
        if self.juego_terminado:
            return 
            
        pistas = self.obtener_percepciones(self.pos_jugador)
        texto = f"Posición: {self.pos_jugador}. "
        if pistas:
            texto += "Sientes: " + ", ".join(pistas)
        else:
            texto += "No sientes nada."
        
        if self.flecha_disponible:
            texto += " [Flecha Lista 🏹]"
        if self.tiene_oro:
            texto += " [¡Tienes el Oro!]"
            
        self.info_label.config(text=texto)

    def mover(self, direccion):
        """Mueve al jugador y evalúa el resultado."""
        if self.juego_terminado:
            return

        r, c = self.pos_jugador
        nueva_pos = self.pos_jugador

        if direccion == "arriba" and r > 0:
            nueva_pos = (r - 1, c)
        elif direccion == "abajo" and r < TAM_CUADRICULA - 1:
            nueva_pos = (r + 1, c)
        elif direccion == "izquierda" and c > 0:
            nueva_pos = (r, c - 1)
        elif direccion == "derecha" and c < TAM_CUADRICULA - 1:
            nueva_pos = (r, c + 1)
        
        if nueva_pos != self.pos_jugador:
            self.pos_jugador = nueva_pos
            self.tablero_visitado.add(self.pos_jugador)
            self.evaluar_estado()
            self.actualizar_canvas()
            self.actualizar_info_label()

    def disparar_flecha(self, direccion):
        """Lógica para disparar la flecha."""
        if self.juego_terminado:
            return
        
        if not self.flecha_disponible:
            self.info_label.config(text="¡Ya no tienes flechas!")
            return

        self.flecha_disponible = False
        
        self.actualizar_estado_botones(tk.NORMAL) 
        
        print(f"Disparando {direccion}...")

        r_jugador, c_jugador = self.pos_jugador
        r_wumpus, c_wumpus = self.pos_wumpus
        
        acierto = False
        if direccion == "arriba" and c_jugador == c_wumpus and r_jugador > r_wumpus:
            acierto = True
        elif direccion == "abajo" and c_jugador == c_wumpus and r_jugador < r_wumpus:
            acierto = True
        elif direccion == "izquierda" and r_jugador == r_wumpus and c_jugador > c_wumpus:
            acierto = True
        elif direccion == "derecha" and r_jugador == r_wumpus and c_jugador < c_wumpus:
            acierto = True
        
        if acierto:
            self.wumpus_vivo = False
            self.info_label.config(text="¡Has oído un grito desgarrador! 😱")
            del self.tablero_logico[self.pos_wumpus] 
            self.actualizar_canvas()
            self.actualizar_info_label() 
        else:
            self.info_label.config(text="La flecha se pierde en la oscuridad...")

    def evaluar_estado(self):
        """Comprueba si el jugador ha muerto o ganado."""
        pos = self.pos_jugador
        peligro = self.tablero_logico.get(pos)

        # 1. Comprobar Muerte
        if peligro == WUMPUS and self.wumpus_vivo:
            self.terminar_juego("¡Has sido devorado por el Wumpus! 💀", "red")
        elif peligro == POZO:
            self.terminar_juego("¡Has caído a un pozo sin fondo! 😵", "red")

        # 2. Comprobar Oro
        if peligro == ORO and not self.tiene_oro:
            self.tiene_oro = True
            print("¡Encontraste el oro!")
            del self.tablero_logico[pos] 

        # 3. Comprobar Victoria
        if self.tiene_oro and self.pos_jugador == (0, 0):
            self.terminar_juego("¡Escapaste con el oro! ¡GANASTE! 🏆", "green")

    def actualizar_estado_botones(self, estado):
        """Activa o desactiva todos los botones."""
        # Botones de Movimiento
        self.btn_arriba.config(state=estado)
        self.btn_abajo.config(state=estado)
        self.btn_izquierda.config(state=estado)
        self.btn_derecha.config(state=estado)
        
        # Botones de Disparo
        estado_disparo = estado
        if not self.flecha_disponible and estado == tk.NORMAL:
            estado_disparo = tk.DISABLED
            
        self.btn_shoot_up.config(state=estado_disparo)
        self.btn_shoot_down.config(state=estado_disparo)
        self.btn_shoot_left.config(state=estado_disparo)
        self.btn_shoot_right.config(state=estado_disparo)

    def terminar_juego(self, mensaje, color_fg):
        """Muestra el mensaje de fin de juego y deshabilita controles."""
        self.juego_terminado = True
        self.info_label.config(text=mensaje, fg=color_fg)
        
        self.actualizar_estado_botones(tk.DISABLED) # Deshabilita todo
        messagebox.showinfo("Fin del Juego", mensaje)

# --- Punto de Entrada Principal ---
if __name__ == "__main__":
    root = tk.Tk()
    app = WumpusWorld(root)
    root.mainloop()

¡Encontraste el oro!
